In [ ]:
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import randperm
from torch._utils import _accumulate
import torchvision
from torchvision import datasets, transforms, models
import xml.etree.ElementTree as ET
import os
from scipy.stats import multivariate_normal
from sklearn.model_selection import train_test_split
import os
import cv2
import torch
import random
import numbers
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch import randperm
from torch._utils import _accumulate
import torchvision
from torchvision import datasets, transforms, models
import torchvision.transforms.functional as F
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn as nn
import time
from skimage import io, transform
import sklearn.metrics as skm
import imutils
from scipy.spatial import distance
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models

Find bounding box given 2 points

In [ ]:
def find_bound_box(values,label,factor):
   xmin = int(int((values.find('xmin').text))*factor/4)
   ymin = int(int((values.find('ymin').text))*factor/4)
   xmax = int(int((values.find('xmax').text))*factor/4)
   ymax = int(int((values.find('ymax').text))*factor/4)
   if label == 0:
     centery = int((ymin+ymax)/2)
   else:
     centery = ymin
   return np.array((xmin,ymin,xmax,ymax,int((xmin+xmax)/2),centery,np.abs(xmax-xmin),np.abs(ymin-ymax))).reshape(-1,2)

Generating the heatmaps given the landmarks of the objects


In [ ]:
def generate_heatMap(label,img_heatmap,bnddtls):
    objects_colors = [[255,255,0],[255,0,255],[0,255,255]]
    radius = [9 ,3 , 15]
    #cyan for ball: 0,255,255 - magenta for goalposts: 255,0,255 - yellow for robot: 255,255,0
    size=radius[label] #aperture
    rad = radius[label]
    x, y = np.meshgrid(np.linspace(-1, 1, rad), np.linspace(-1, 1, rad))
    d = np.sqrt(x * x + y * y)
    sigma, mu = 1.0, 0.0
    kernel = np.exp(-((d - mu) ** 2 / (2.0 * sigma ** 2)))

    uniq = np.unique(kernel)
    halfsize = int(uniq.shape[0] /2)
    mean = uniq[halfsize]

    size = int(size/2)

    if bnddtls[2][1].item()+size > img_heatmap.shape[0]-1:
                y_begin = img_heatmap.shape[0]-1-size
    elif bnddtls[2][1].item()-size < 0:
                y_begin = bnddtls[2][1] + size
    else:
                y_begin = int(bnddtls[2][1].item())

    if bnddtls[2][0].item()+size > img_heatmap.shape[1]-1:
                x_begin = img_heatmap.shape[1]-1-size
    elif bnddtls[2][0].item()-size < 0:
                x_begin = bnddtls[2][0] + size

    else:
                x_begin = int(bnddtls[2][0].item())

    heatmap_value = img_heatmap[(y_begin -size) : (y_begin + (size+1)), (x_begin - size) : (x_begin + (size+1))]

    heatmap_value[kernel >= mean] = objects_colors[label]
    
    img_heatmap[(y_begin -size) : (y_begin + (size+1)), (x_begin - size) : (x_begin + (size+1))] = heatmap_value
     
    return img_heatmap

Creating the heatmaps: 1- reading all the images and lanmarks of the detection dataset. 
2- calling the generating_heatmap function to generate heatmap for each image.
3- Save the generated heatmap in the dataset path

In [ ]:
def create_heatMap_images(root_dir):
  labels = {"ball":0 , "goalpost":1 , "robot":2}

  k = 0
  for file in os.listdir(root_dir):
    if file.endswith(".jpg"):
      extension = ".jpg"
    elif file.endswith(".jpeg"):
      extension = ".jpeg"
    elif file.endswith(".png") and not file.endswith("_heatmap.png"):
      extension = ".png"
    else:
      continue

    filepath = os.path.join(root_dir,file)

    xml_file = filepath.replace(extension,'.xml')
    filename = filepath.replace(extension,'_heatmap.png')

    if os.path.isfile(xml_file):
      if (file.startswith('b')):
        factor = 0.8
      else:
        factor =1
      image = cv2.imread(filepath,cv2.COLOR_BGR2RGB)
      new_shape = (int(image.shape[0]/4),int(image.shape[1]/4),3)
      img_heatmap = np.ones(new_shape) * 255
      xml_data = ET.parse(xml_file).getroot()

      all_dtls= []

      all_objects_list = xml_data.findall('object')

      for group in all_objects_list:
          bndvalues = group.find('bndbox')
          name = group.find('name').text
          label = labels[name]
          land_marks = find_bound_box(bndvalues,label,factor)
          img_heatmap = generate_heatMap(label,img_heatmap,land_marks)
          
      cv2.imwrite(filename, img_heatmap)
      k+=1
      
  print("done ", k)  

This function is run before the training to save all the heatmaps in the dataset path, to save the time of generating the heatmaps during training phase

In [ ]:
#create_heatMap_images('/content/drive/My Drive/detection/dataset')

Defining the global variables that are needed in the Custom Dataset Classes


In [ ]:
resize_mode = "resize"
resize_factor = 1
mode = ""
trs = [transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                 transforms.ToTensor()]

Custom Dataset for  Detection Dataset


In [ ]:
class DetectionDataset(Dataset):
    """Cutomized Dataset used to train the model.
    Args:
      root_dir: path where all the training files are saved.
      transform: transformations to be applied to the dataset.
      filenames: names of all the files in training dataset.
    """
    def __init__(self, root_dir, transformm=None):
        super(DetectionDataset, self).__init__()
        self.root_dir = root_dir
        global trs
        self.filenames = []
        self.heatmap_filenames = []
        self.xml_filenames = []
        global resize_mode
        global resize_factor
        global mode 

        for file in os.listdir(root_dir):
            filepath = os.path.join(root_dir,file)
            
            if file.endswith(".jpg"):
              extension = ".jpg"
            elif file.endswith(".jpeg"):
              extension = ".jpeg"
            elif file.endswith(".png") and not file.endswith("_heatmap.png"):
              extension = ".png"
            else:
              continue

            heatmap_file_path = filepath.replace(extension,"_heatmap.png")

            if os.path.isfile(heatmap_file_path):
              xml_file = file.replace(extension,'.xml')
              heatmapfile = file.replace(extension,"_heatmap.png")
              self.heatmap_filenames.append(heatmapfile)
              self.filenames.append(file)
              self.xml_filenames.append(xml_file)

    def __len__(self):
        """Length of the dataset"""
        return len(self.heatmap_filenames)

    def __getitem__(self, ind):

        filepath = os.path.join(self.root_dir,self.filenames[ind])
        heatmap_path = os.path.join(self.root_dir,self.heatmap_filenames[ind])

        img = Image.open(filepath)
        heatmap = Image.open(heatmap_path)
        inputImage = img.copy()

        if(resize_mode=="resize"):
          img = img.resize((int(64*resize_factor),int(32*resize_factor)),Image.NEAREST)
          heatmap = heatmap.resize((int(16*resize_factor),int(8*resize_factor)),Image.NEAREST)

        if trs:
            if type(trs) is not list:
              transform = [trs]

            transform_list_len = len(trs)
            
            for idx in range(transform_list_len):
              img = trs[idx](img)     
            
            heatmap = F.to_tensor(heatmap)
            heatmap[heatmap == 0] = 0.1
            inputImage = F.to_tensor(inputImage)           
        
        return {'input':inputImage,'image': img, 'heatmap': heatmap}

Custom Dataset for  Segmentation Dataset


In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, folder_path,transform=False):
        super(SegmentationDataset, self).__init__()
        self.root_dir = folder_path
        self.filenames = []
        self.target_filenames=[]
        global resize_mode
        global resize_factor
        global mode 
        global trs

        file_path = os.path.join(self.root_dir,"image")

        for file in os.listdir(file_path):
            if file.endswith(".jpg"):
              extension = ".jpg"
            elif file.endswith(".jpeg"):
              extension = ".jpeg"
            elif file.endswith(".png"):
              extension = ".png"
            else:
              continue

            targetfile = file.replace(extension,".png")
            target_filepath = os.path.join(self.root_dir,"target",targetfile)

            if os.path.isfile(target_filepath):
              self.target_filenames.append(targetfile)
              self.filenames.append(file)

        

    def __getitem__(self, index):
    
            img_path = os.path.join(self.root_dir,"image",self.filenames[index])
            mask_path = os.path.join(self.root_dir,"target",self.target_filenames[index])
          
            img = Image.open(img_path)
            label = cv2.imread(mask_path,0)
            inputImage = img.copy()

            if(resize_mode == "resize"):
              img = img.resize((int(64*resize_factor),int(32*resize_factor)),Image.NEAREST)
              label = Image.fromarray(np.uint8(label))
              label = label.resize((int(16*resize_factor),int(8*resize_factor)),Image.NEAREST)

            if trs:
              length_transform = len(trs)
              
              for idx in range(length_transform):
                img = trs[idx](img)
                                            
            inputImage = F.to_tensor(inputImage)
            label = F.to_tensor(label)
            target = torch.zeros_like(label)
            target[(label.min() == label)] = 0
            target[((label.min() < label) & (label.max() > label))] = 1
            target[(label.max() == label)] = 2
            target = target.squeeze()
            dataset = {'input':inputImage ,'image': img, 'target': target}
            return dataset

    def __len__(self):
        return len(self.filenames)

 Implmentatation of Niloofar Azizi*, Hafez Farazi*, and Sven Behnke:
Location Dependency in Video Prediction   https://github.com/AIS-Bonn/LocDepVideoPrediction/blob/master/Conv_PGP_LB.ipynb

In [ ]:
class LocationAwareConv2d(torch.nn.Conv2d):
    def __init__(self,lb,gradient,w,h,in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super().__init__(in_channels, out_channels, kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.locationBias=lb
        self.locationEncode=torch.autograd.Variable(torch.ones(w,h,3))
        if gradient:
            for i in range(w):
                self.locationEncode[i,:,1]=self.locationEncode[:,i,0]=(i/float(w-1))
    def forward(self,inputs):
        if self.locationBias.device != inputs.device:
            self.locationBias=self.locationBias.to(inputs.get_device())
        if self.locationEncode.device != inputs.device:
            self.locationEncode=self.locationEncode.to(inputs.get_device())
        b=self.locationBias*self.locationEncode

        return super().forward(inputs)+b[0:inputs.shape[2],0:inputs.shape[3],0]+b[0:inputs.shape[2],0:inputs.shape[3],1]+b[0:inputs.shape[2],0:inputs.shape[3],2]

The Decoder block in NimbroNet2

In [ ]:
def decoder_block(block_number, input_channels, output_channels=0,kernel=0,stride=0, padding=0, output_padding=0):
  if block_number == 1:
    block = nn.Sequential(
      nn.ReLU(),
      nn.ConvTranspose2d(input_channels,output_channels,kernel,stride, padding,output_padding)
      )
  elif block_number == 4:
    block = nn.Sequential(
      nn.ReLU(),
      nn.BatchNorm2d(input_channels),
      )
  else:
    block = nn.Sequential(
      nn.ReLU(),
      nn.BatchNorm2d(input_channels),
      nn.ConvTranspose2d(input_channels,output_channels,kernel,stride, padding,output_padding)
      )
  return block

NimbroNet2 Model

In [ ]:
PAD = 0
class NimbroNet2(nn.Module):
  def __init__(self):
    super(NimbroNet2,self).__init__()
    # Encoder 
    resnet_encoder = models.resnet18(pretrained=True)
    del resnet_encoder.avgpool
    del resnet_encoder.fc
    # Model Encoders (Resnet)
    self.resnet_encoder_block1 = nn.Sequential(*list(resnet_encoder.children())[0:5])
    self.resnet_encoder_block2 = nn.Sequential(*list(resnet_encoder.children())[5:6])
    self.resnet_encoder_block3 = nn.Sequential(*list(resnet_encoder.children())[6:7])
    self.resnet_encoder_block4 = nn.Sequential(*list(resnet_encoder.children())[7:8]) # 512

    # Model Decoders
    self.decoder_block_1 = decoder_block(1, 512, 256, 2, 2, 0, 0)
    self.decoder_block_2 = decoder_block(2, 512, 256, 2, 2, 0, 0)
    self.decoder_block_3 = decoder_block(3, 512, 128, 2, 2, 0, 0)
    self.decoder_block_4 = decoder_block(4, 256)

    #intermediate convolutions
    self.conv_en3_de1 = nn.Conv2d(256, 256, kernel_size=1)   
    self.conv_en2_de2 = nn.Conv2d(128, 256, kernel_size=1) 
    self.conv_en1_de3 = nn.Conv2d(64, 128, kernel_size=1) 
    self.locationBias = torch.nn.Parameter(torch.zeros(120,160,3))
    
    #location dependant bias
    self.lbS = LocationAwareConv2d(self.locationBias,True,120+(PAD*2),160+(PAD*2),in_channels=256, out_channels=3, kernel_size=1, stride=1, padding=(1-2)//2+1, bias= True)
    self.lbD = LocationAwareConv2d(self.locationBias,True,120+(PAD*2),160+(PAD*2),in_channels=256, out_channels=3, kernel_size=1, stride=1, padding=(1-2)//2+1, bias= True)

  def forward(self,x):
    x = self.resnet_encoder_block1(x)
    intermediate_output_1 = self.conv_en1_de3(x)
    x = self.resnet_encoder_block2(x)
    intermediate_output_2 = self.conv_en2_de2(x)
    x = self.resnet_encoder_block3(x)
    intermediate_output_3 = self.conv_en3_de1(x)
    x = self.resnet_encoder_block4(x)
    x = self.decoder_block_1(x)
    decoder_block_2_input = torch.cat((x,intermediate_output_3),1)
    decoder_block_2_output = self.decoder_block_2(decoder_block_2_input)
    decoder_block_3_input = torch.cat((decoder_block_2_output,intermediate_output_2),1)
    decoder_block_3_output = self.decoder_block_3(decoder_block_3_input)
    decoder_block_4_input = torch.cat((decoder_block_3_output,intermediate_output_1),1)
    decoder_block_4_output_0 = self.decoder_block_4(decoder_block_4_input)
    decoder_block_4_output_1 = self.lbS(decoder_block_4_output_0)
    decoder_block_4_output_2 = self.lbD(decoder_block_4_output_0)
    return decoder_block_4_output_1,decoder_block_4_output_2
  

Defining the criterion for each head and the optimizer



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = NimbroNet2()
model.to(device)

# Criterion for Detection
criterion_d = nn.MSELoss()
criterion_d.to(device)

# Criterion for Segmentation
softmaxing = nn.LogSoftmax(dim=1)
criterion_s = nn.NLLLoss()
criterion_s.to(device)

# optimizer
optimizer = optim.AdamW(
                        [{"params":model.resnet_encoder_block1.parameters(), "lr": 0.0000001,'weight_decay':10e-12},
                        {"params":model.resnet_encoder_block2.parameters(), "lr": 0.0000001,'weight_decay':10e-12},
                        {"params":model.resnet_encoder_block3.parameters(), "lr": 0.0000001,'weight_decay':10e-12},
                        {"params":model.resnet_encoder_block4.parameters(), "lr": 0.0000001,'weight_decay':10e-12},
                        {"params":model.decoder_block_1.parameters(), "lr": 0.0001,'weight_decay':0},
                        {"params":model.decoder_block_2.parameters(), "lr": 0.0001,'weight_decay':0},
                        {"params":model.decoder_block_3.parameters(), "lr": 0.0001,'weight_decay':0},
                        {"params":model.decoder_block_4.parameters(), "lr": 0.0001,'weight_decay':0},
                        {"params":model.conv_en3_de1.parameters(), "lr": 0.0001,'weight_decay':0},
                        {"params":model.conv_en2_de2.parameters(), "lr": 0.0001,'weight_decay':0},
                        {"params":model.conv_en1_de3.parameters(), "lr": 0.0001,'weight_decay':0},
                        {"params":model.locationBias, "lr": 0.0001,'weight_decay':0}], lr=0.0001,weight_decay=0)

# optimizer scheduler for Cyclic learning rate
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.0001, max_lr=0.0006,step_size_up = 20000,cycle_momentum=False)

Detection Datasets & Dataloaders


In [ ]:
                                 ##### Detection Datasets & Dataloaders #####

# Augmentation for train and Validation
transforms_train = [transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                    transforms.ToTensor()]

# Augmentation for Testing
transforms_test =[transforms.ToTensor()]
                
##### Detection Dataset ####

# batchsize for detection
batch_size_detection = 30

# The whole dataset for detection
dataset_detection = DetectionDataset('/content/drive/My Drive/detection/dataset',transforms_train)


### spliting the dataset into 80% for training and 10% validation and 10% testing ###

# counting the needed lengths for detection datasets
len_dataset_detection = len(dataset_detection)
len_train_dataset_detection,len_rest_dataset_detection = int(len_dataset_detection * 0.8),int(len_dataset_detection * 0.2)
# add the difference that lost in the division process to the rest of the dataset
len_rest_dataset_detection += len_dataset_detection - (len_train_dataset_detection+len_rest_dataset_detection)
len_valid_dataset_detection , len_test_dataset_detection = int(len_rest_dataset_detection*0.5),int(len_rest_dataset_detection*0.5)
# add the difference that lost in the division process to the validationdataset
len_valid_dataset_detection += len_rest_dataset_detection - (len_test_dataset_detection+len_valid_dataset_detection)

### splitting datasets ###
train_dataset_detection,evaluation_dataset_detection = torch.utils.data.random_split(dataset_detection,[len_train_dataset_detection,len_rest_dataset_detection])
validation_dataset_detection,test_dataset_detection = torch.utils.data.random_split(evaluation_dataset_detection,[len_valid_dataset_detection,len_test_dataset_detection])

### dataloaders for detection ###
train_loader_detection = torch.utils.data.DataLoader(dataset=train_dataset_detection, 
                                           batch_size=batch_size_detection, 
                                           shuffle=True)#,**kwargs)
val_loader_detection = torch.utils.data.DataLoader(dataset=validation_dataset_detection, 
                                           batch_size=batch_size_detection, 
                                           shuffle=True)#,**kwargs)
test_loader_detection = torch.utils.data.DataLoader(dataset=test_dataset_detection, 
                                           batch_size=batch_size_detection, 
                                           shuffle=True)#,**kwargs)

print("for Detection: ", len_train_dataset_detection, " samples for training ", len_valid_dataset_detection, " samples for validation ", len_test_dataset_detection, " samlpes for testing")

Segmentation Datasets & Dataloaders

In [ ]:
                                ##### Segmentation Datasets & Dataloaders #####

# Augmentation for segmentation training and validation
transforms_train_segmentation = [
                 transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                 transforms.ToTensor()
                 ]

#### segmentation dataset #####

# batch size for segmentation
batch_size_segmentation = 30

# The whole dataset for segmentation
dataset_segmentation = SegmentationDataset('/content/drive/My Drive/dataset',transforms_train_segmentation)


### spliting the dataset into 80% for training and 10% validation and 10% testing ###

# counting the needed lengths for segmentation datasets
len_dataset_segmentation = len(dataset_segmentation)
len_train_dataset_segmentation,len_rest_dataset_segmentation = int(len_dataset_segmentation * 0.8),int(len_dataset_segmentation * 0.2)
len_rest_dataset_segmentation += len_dataset_segmentation - (len_train_dataset_segmentation+len_rest_dataset_segmentation)
len_valid_dataset_segmentation,len_test_dataset_segmentation = int(len_rest_dataset_segmentation*0.5),int(len_rest_dataset_segmentation*0.5)
len_valid_dataset_segmentation += len_rest_dataset_segmentation - (len_test_dataset_segmentation+len_valid_dataset_segmentation)

# splitting datasets
train_dataset_segmentation,evaluation_dataset_segmentation = torch.utils.data.random_split(dataset_segmentation,[len_train_dataset_segmentation,len_rest_dataset_segmentation])
validation_dataset_segmentation,test_dataset_segmentation = torch.utils.data.random_split(evaluation_dataset_segmentation,[len_valid_dataset_segmentation,len_test_dataset_segmentation])

### dataloaders ###
train_loader_segmentation = torch.utils.data.DataLoader(dataset=train_dataset_segmentation, 
                                           batch_size=batch_size_segmentation, 
                                           shuffle=True)#,**kwargs)
val_loader_segmentation = torch.utils.data.DataLoader(dataset=validation_dataset_segmentation, 
                                           batch_size=batch_size_segmentation, 
                                           shuffle=True)#,**kwargs)
test_loader_segmentation = torch.utils.data.DataLoader(dataset=test_dataset_segmentation, 
                                           batch_size=batch_size_segmentation, 
                                           shuffle=True)#,**kwargs)

print("for Segmentation: ", len_train_dataset_segmentation, " samples for training ", len_valid_dataset_segmentation, " samples for validation ", len_test_dataset_segmentation, " samlpes for testing")

total variational loss computaion function

In [ ]:
def tv_loss(img,loss_type):

    tv_loss_weight = 2e-5 # is the total variation loss weight
    loss= torch.zeros_like(loss_type)
    #loss.requires_grad(False)
    for i in range(img.shape[0]):
      # Computing the loss for the first channel (channel for background: class 0)
      w_variance0 = torch.sum((img[i,0,:,1:] - img[i,0,:,:-1])**2)
      h_variance0 = torch.sum((img[i,0,1:,:] - img[i,0,:-1,:])**2)
      loss += tv_loss_weight * (w_variance0 + h_variance0)

      # Computing the loss for the second channel (channel for field: class 1)
      w_variance1 = torch.sum((img[i,1,:,1:] - img[i,1,:,:-1])**2)
      h_variance1 = torch.sum((img[i,1,1:,:] - img[i,1,:-1,:])**2)
      loss += tv_loss_weight * (w_variance1 + h_variance1)
    
    return loss


Train model function

In [ ]:
trans = transforms.ToPILImage()
def train_model(model,train_loader, criterion, optimizer, seg,loss_vector,log_interval):
    images = None
    heatmaps = None
    for j,train_data in enumerate(train_loader):
        
        images = train_data['image'].to(device)
        
        # model is in train mode
        model.train()
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output
        outputsS,outputsD = model(images)


        ### Calculate Loss ###
        if seg:
          target = train_data['target'].to(device)
          #print(target.shape)
          target = target.long()
          loss = criterion(softmaxing(outputsS), target)
          loss += tv_loss(outputsS,loss)
  
        else:
          heatmaps = train_data['heatmap'].to(device)
          outputsD = outputsD.float()
          loss = criterion(outputsD, heatmaps)
          #train_loss += loss

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        # chaning the learning rate of the cyclic learning rate
        scheduler.step()

        if j % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, j * len(images), len(train_loader.dataset),
                100. * float(j) / len(train_loader), loss.item()))
            loss_vector.append(loss)

    # showing the output after training for one epoch        
    outputsS,outputsD = model(images)
    
    if seg == False:
      outputsD[outputsD<0] = 0
      outputsD[outputsD>1] = 1
      plt.imshow(trans(outputsD[0].cpu()))
      plt.show()
      plt.imshow(trans(heatmaps[0].cpu()))
      plt.show()

    if seg:
      img = outputsS.data.max(1)[1]        
      img[img == 0] = 0
      img[img == 2] = 255 
      img[img == 1] = 128
      
      plt.imshow(img[0].cpu(), cmap='gray', vmin=0, vmax=255,interpolation='nearest')
      plt.show()

    plt.imshow(transforms.ToPILImage()(images[0].cpu()))
    plt.show()

    print("lr = ",optimizer.state_dict()["param_groups"][0]["lr"])
    return loss_vector

Freezing function : to freeze/unfreeze the given parameters in the model

In [ ]:
def freezing(parameters,value):
  for param in parameters:
    param.requires_grad = value

In [ ]:
# 2 arrays to monitor the loss of the detection and segmentation
loss_d = []
loss_s = []

The first 50 epochs of training: 
the samples are downsampled, and then upsampled gradually. The parameters of the encoeder are frozen. Checkpoints are saved after every 25 epochs

In [ ]:
# for first 50 epochs freez the Resnet Encoder
freezing(model.resnet_encoder_block1.parameters(),False)
freezing(model.resnet_encoder_block2.parameters(),False)
freezing(model.resnet_encoder_block3.parameters(),False)
freezing(model.resnet_encoder_block4.parameters(),False)

# model is in train mode
model.train()

### for first 50 epochs resizing the input and target images! ###
resize_mode="resize"

# number of epochs for resizing
epochs = 50

for epoch in range(1, epochs+1):
    print("epoch",epoch)

    # resizing factor to upsample the samples gradually
    resize_factor = int((epoch/8)+1)

    loss_s = train_model(model,train_loader_segmentation, criterion_s, optimizer,True,loss_s,10)

    loss_d = train_model(model,train_loader_detection, criterion_d, optimizer,False,loss_d,20)
    
    if epoch % 25 == 0:
      path = "/content/drive/My Drive/Final Model Checkpoints/final_whole_mode_withgreatergoalpost_Checkpoint_epoch" + str(epoch) + ".th"
      torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_d':loss_d,
            'loss_s':loss_s,
            'DetectionTrainDataLoader': train_loader_detection,
            'SegmentationTrainDataLoader': train_loader_segmentation,
            'DetectionTestDataLoader': test_loader_detection,
            'DetectionValidationDataLoader': val_loader_detection,
            'SegmentationTestDataLoader': test_loader_segmentation,
            'SegmentationValidationDataLoader': val_loader_segmentation,
            'resize_factor':resize_factor,
            }, path)

The next 50 epochs in the training: Fully-sized images are trained and the paramerter of the encoder are unfrozen (the model is trained jointly)

In [ ]:
# for Next 50 epochs unfreez the Resnet Encoder
freezing(model.resnet_encoder_block1.parameters(),True)
freezing(model.resnet_encoder_block2.parameters(),True)
freezing(model.resnet_encoder_block3.parameters(),True)
freezing(model.resnet_encoder_block4.parameters(),True)

# for only first epoch we resize the input and target images because it some time take a large capacity from the GPU!
# then we continue with images full-sizes

# model is in train mode
model.train()

resize_mode="resize"
resize_factor = 4

epochs = 150
epoch_init = 50

#to be returned from the model training
for epoch in range(epoch_init+1, epochs+1):
    print("epoch",epoch)

    if epoch > (epoch_init+1):
      resize_mode = ""
      resize_factor = 1

    loss_d = train_model(model,train_loader_detection, criterion_d, optimizer,False,loss_d,20)
    loss_s = train_model(model,train_loader_segmentation, criterion_s, optimizer,True,loss_s,10)
    
    if epoch % 25 == 0:
      path = "/content/drive/My Drive/Final Model Checkpoints/final_whole_mode_withsmallergoalpost_Checkpoint_epoch" + str(epoch) + ".th"
      torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss_d':loss_d,
            'loss_s':loss_s,
            'DetectionTrainDataLoader': train_loader_detection,
            'SegmentationTrainDataLoader': train_loader_segmentation,
            'DetectionTestDataLoader': test_loader_detection,
            'DetectionValidationDataLoader': val_loader_detection,
            'SegmentationTestDataLoader': test_loader_segmentation,
            'SegmentationValidationDataLoader': val_loader_segmentation,
            'resize_factor':1,
            }, path)

In [ ]:
path = "/content/drive/My Drive/Final Model Checkpoints/Checkpoint_epoch125.th"
checkpoint = torch.load(path)#, map_location=lambda storage, loc: storage)
#print(checkpoint['model_state_dict'])
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch_init = checkpoint['epoch']
#train_loader_alltogether = checkpoint['train_loader_alltogether']
test_loader_detection = checkpoint['DetectionTestDataLoader']
val_loader_detection = checkpoint['DetectionValidationDataLoader']
test_loader_segmentation = checkpoint['SegmentationTestDataLoader']
val_loader_segmentation = checkpoint['SegmentationValidationDataLoader']
train_loader_detection = checkpoint['DetectionTrainDataLoader']
train_loader_segmentation = checkpoint['SegmentationTrainDataLoader']
loss_d = checkpoint['loss_d']
loss_s = checkpoint['loss_s']

Ploting the learning rate curve for detection and segmentation

In [ ]:
# learning rate for detection
plt.plot(range(0,len(loss_d)), loss_d)
plt.title("learning rate for detection")
plt.ylabel('Train loss')
plt.xlabel('No of Epochs')
plt.show()

# learning rate for segmentation
plt.plot(range(0,len(loss_s)), loss_s)
plt.title("learning rate for segmentation")
plt.ylabel('Train loss')
plt.xlabel('No of Epochs')
plt.show()

In [ ]:
# transformation for testing
trs = [transforms.ToTensor()]

Computing the testing accuracy metrics for segmentation

In [ ]:
resize_mode= ""
model.eval()
k=1
SMOOTH = 1e-6
lossv, accv = [], []
total_train = 0
correct_train = 0
acc0 = []
acc1 = []
acc2 = []
iou0 = []
iou1 = []
iou2 = []
for j,train_data in enumerate(test_loader_segmentation):
      k +=1

      images = train_data['image'].to(device)
      images2 = train_data['input'].to(device)
      labels = train_data['target'].to(device)
      images.requires_grad_(False)
      images2.requires_grad_(False)
      labels.requires_grad_(False)
      
      output,_ = model(images)

      for i in range(output.shape[0]):
        a = output[i].argmax(0)
        _, predicted = torch.max(output.data, 1)
        b = predicted.cpu().detach().numpy()
        a = labels.cpu().detach().numpy() 
        pred_0 = np.count_nonzero(b == 0)
        label_0 = np.count_nonzero(a == 0)
        intersection_0 = sum((a==0)[b==0])
        acc0.append(intersection_0 / label_0)
        union_0 = pred_0 + label_0 - intersection_0
        iou = (intersection_0 + SMOOTH) / (union_0 + SMOOTH)
        iou0.append(iou)

        b = predicted.cpu().detach().numpy()
        a = labels.cpu().detach().numpy() 
        pred_1 = np.count_nonzero(b == 1)
        label_1 = np.count_nonzero(a == 1)
        intersection_1 = sum((a==1)[b==1])
        acc1.append(intersection_1 / label_1)
        union_1 = pred_1 + label_1 - intersection_1
        iou = (intersection_1 + SMOOTH) / (union_1 + SMOOTH)
        iou1.append(iou)

        b = predicted.cpu().detach().numpy()
        a = labels.cpu().detach().numpy() 
        pred_2 = np.count_nonzero(b == 2)
        label_2 = np.count_nonzero(a == 2)
        intersection_2 = sum((a==2)[b==2])
        acc2.append(intersection_2 / label_2)
        union_2 = pred_2 + label_2 - intersection_2
        iou = (intersection_2 + SMOOTH) / (union_2 + SMOOTH)
        iou2.append(iou)
      
      output = output.data.max(1)[1]

      for i in range(output.shape[0]):

        plt.title('Test image' + str(k))
        image = np.uint8(np.asanyarray(transforms.ToPILImage()(images2[i].cpu())))

        plt.imshow(image)
        plt.show()

        img = output[i].cpu()
        img[img == 0] = 0
        img[img == 2] = 255 
        img[img == 1] = 128
        
        plt.imshow(img, cmap='gray', vmin=0, vmax=255,interpolation='nearest')
        plt.show()


Calculating the accuracy for each segmented class

In [ ]:
resize_mode=""
#Calculating accuracy for each segmented class
acc00=0
acc11=0 
acc22 =0
for i in acc0:
  acc00 += i
acc00 /= len(acc0) 
print('Accuracy for background is' , acc00*100)

for i in acc1:
  acc11 += i
acc11 /= len(acc1) 
print('Accuracy for field is' , acc11*100)

for i in acc2:
  acc22 += i
acc22 /= len(acc2) 
print('Accuracy for lines is' , acc22*100)

Calculating the IOU for each segmented class

In [ ]:
resize_mode=""
# Calculating IOU for each segmented class
iou00=0
iou11=0 
iou22 =0
for i in iou0:
  #print(i)
  iou00 += i
iou00 /= len(iou0) 
print('IOU for background is' , iou00)

for i in iou1:
  #print(i)
  iou11 += i
iou11 /= len(iou1) 
print('IOU for field is' , iou11)

for i in iou2:
  iou22 += i
iou22 /= len(iou2) 
print('IOU for lines is' , iou22)

Find the center of the contour to get the centers of detected objects https://www.pyimagesearch.com/2016/02/01/opencv-center-of-contour/#download-the-code

In [ ]:
def find_center_contour(image):
  
  ball_centers = []
  goalpost_centers = []
  robot_centers = []

  # converting image to grayscale image
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  
  # blurring this grayscale image
  blurred = cv2.GaussianBlur(gray, (3, 3), 2)

  # convert the grayscale image to binary image
  ret,thresh = cv2.threshold(blurred,250,255,0)

  # find contours in the binary image
  contours, hierarchy = cv2.findContours(thresh.astype(np.uint8),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

  for c in contours:
    # calculate moments for each contour
    M = cv2.moments(c)

    # calculate x,y coordinate of center
    if M["m00"] != 0:
      cX = int(M["m10"] / M["m00"])
    else:
      cX = 100000
    if M["m00"] != 0:
      cY = int(M["m01"] / M["m00"])
    else:
      cY = 100000
    
    if (cX != 100000) and (cY != 100000):
      # if it has a ball color
      if image[cY,cX,0] < 200:     
        ball_centers.append((cX,cY))
      # if it has goalpost color
      elif image[cY,cX,1] < 200:
        goalpost_centers.append((cX,cY))
      #if it has robot color
      elif image[cY,cX,2] < 200:
        robot_centers.append((cX,cY))

  return ball_centers,goalpost_centers,robot_centers

Computing the confusion matrix for the detection accuracy metrics

In [ ]:
def get_confusion_matrix(model_output_centers, ground_truth_centers):
  radius = 10 #pixels
  TP = 0
  FN = 0
  FP = 0
  
  gt_length = len(ground_truth_centers)
  model_length = len(model_output_centers)

  objects_detected = np.zeros(gt_length)
  
  for k in range(model_length):
    found = -1
    for l in range(gt_length):

        dist = distance.euclidean(model_output_centers[k],ground_truth_centers[l])
        if dist <= radius:
          TP +=1
          found = l
          break
    if found == -1: #if found is not changed => object is detected by the model but not in the ground truth 
      FP += 1 
    else:
      objects_detected[found] = 1
  
  # False negative is the number of groundtruth that are not detected by the model
  FN = np.count_nonzero(objects_detected == 0)
  
  return TP, FP, FN

Computig the accuracy for detectio metrics

In [ ]:
resize_mode=""
TP_all = torch.zeros(3)
FP_all = torch.zeros(3)
FN_all = torch.zeros(3)
recall = torch.zeros(3)
precision = torch.zeros(3)
fdr = torch.zeros(3)
accuracy = torch.zeros(3)
f1 = torch.zeros(3)
topil = transforms.ToPILImage()

model.eval()
for j,test_data in enumerate(test_loader_detection):

      images = test_data['image'].to(device)
      labels = test_data['heatmap'].to(device)
      images.requires_grad_(False)
      labels.requires_grad_(False)
      
      _,output = model(images)
      output[output<=0.6] = 0
      output[output>1] =1

      for i in range(output.shape[0]):

        output_ball_centers,output_goalpost_centers,output_robot_centers = find_center_contour(np.asarray(topil(output[i].cpu())))

        ground_truth_ball_centers,ground_truth_goalpost_centers,ground_truth_robot_centers = find_center_contour(np.asarray(topil(labels[i].cpu())))
        
        # get confussion matrix for ball
        TP, FP, FN = get_confusion_matrix(output_ball_centers, ground_truth_ball_centers)

        TP_all[0] += TP 
        FP_all[0] += FP 
        FN_all[0] += FN
        
        # get confussion matrix for goalpost
        TP, FP, FN = get_confusion_matrix(output_goalpost_centers, ground_truth_goalpost_centers)
        TP_all[1] += TP 
        FP_all[1] += FP 
        FN_all[1] += FN 

        # get confussion matrix for robot
        TP, FP, FN = get_confusion_matrix(output_robot_centers, ground_truth_robot_centers)
        TP_all[2] += TP 
        FP_all[2] += FP 
        FN_all[2] += FN 

# Calculating the accuracy metrics
for i in range(3):

  if (FP_all[i] + TP_all[i] + FN_all[i] ) != 0:
   accuracy[i] = TP_all[i] / (FP_all[i] + TP_all[i] + FN_all[i])
  
  if (TP_all[i] + FN_all[i] ) != 0:
    recall[i] = TP_all[i] / (TP_all[i] + FN_all[i] )
  
  if (TP_all[i] + FP_all[i]) != 0:
    precision[i] = TP_all[i] / (TP_all[i] + FP_all[i])
    fdr[i] = FP_all[i] / (TP_all[i] + FP_all[i])
  
  if (precision[i] + recall[i]) != 0:
    f1[i] = 2 * (precision[i] * recall[i]) / (precision[i] + recall[i])

print("Accuracy for ball", accuracy[0])       
print("Accuracy for goal post", accuracy[1])
print("Accuracy for robot", accuracy[2])

print("Precision for ball", precision[0])     
print("Precision for goal post", precision[1])
print("Precision for robot", precision[2])

print("Recall for ball", recall[0])    
print("Recall for goal post", recall[1])   
print("Recall for robot", recall[2])   

print("FDR for ball", fdr[0])         
print("FDR for goal post", fdr[1])   
print("FDR for robot", fdr[2])   

print("F1 for ball", f1[0])      
print("F1 for goal post", f1[1])  
print("F1 for robot", f1[2])  

Computing the validation loss detection

In [ ]:
# calculating the validation loss for detection
resize_mode=""
Validation_loss = 0.0

trans = transforms.ToPILImage()

for j,train_data in enumerate(val_loader_detection):
        
        images = train_data['image'].to(device)
        heatmaps = train_data['heatmap'].to(device)

        images.requires_grad_(False)
        heatmaps.requires_grad_(False)
        
        # model is in validation mode
        model.eval()
        
        
        # Forward pass to get output
        _,outputsD = model(images)
    
        outputsD[outputsD<=0.6] = 0
        outputsD[outputsD>1] = 1

        plt.imshow(trans(outputsD[0].cpu()))
        plt.show()
        plt.imshow(transforms.ToPILImage()(images[0].cpu()))
        plt.show()
        loss = criterion_d(outputsD, heatmaps)
        Validation_loss += loss

print(" Validation loss for detection = ",Validation_loss/len(val_loader_detection))

Computing the validation loss for segmentation

In [ ]:
resize_mode=""
# calculating the validation loss for segmentation
Validation_loss = 0.0

trans = transforms.ToPILImage()

for j,train_data in enumerate(val_loader_segmentation):
        
        images = train_data['image'].to(device)
        heatmaps = train_data['heatmap'].to(device)

        images.requires_grad_(False)
        heatmaps.requires_grad_(False)
        
        # model is in validation mode
        model.eval()
        
        
        # Forward pass to get output
        outputsS,_ = model(images)
    
        img = outputsS.data.max(1)[1]        
        img[img == 0] = 0
        img[img == 2] = 255 
        img[img == 1] = 128
        
        plt.imshow(img[0].cpu(), cmap='gray', vmin=0, vmax=255,interpolation='nearest')
        plt.show()

        plt.imshow(transforms.ToPILImage()(images[0].cpu()))
        plt.show()

        loss = criterion_s(outputsS, heatmaps)
        Validation_loss += loss

print("validation loss for segmentation = ",Validation_loss/len(val_loader_detection))